# Global Inequalities in Cervical Cancer Rates
## Data Analytics and Visualization Project
## by AR


### Data Loading, Preprocessing and Mining

In [1]:
# importing necessary libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sqlite3
import requests
from bs4 import BeautifulSoup
import seaborn as sns
import pycountry
from pycountry import countries
import scipy.stats as st
import plotly.express as px
import plotly.offline as pyo
import plotly.graph_objs as go
import plotly.io as pio

sns.set_palette("bright")
pio.templates.default = "plotly_white"
%matplotlib inline

In [2]:
# load first dataset with CC incidence rates
df_incid_countries = pd.read_csv('cc_incidence_countries_2020_iarc.csv', encoding='latin-1', index_col=False)
df_incid_countries

,Population,Number,Uncertainty interval,Crude Rate,ASR (World),Cum. risk
0,World,604127,[582031.0-627062.0],15.6,13.3,1.82
1,India,123907,[119237.0-128760.0],18.7,18.0,3.12
2,China,109741,[106978.0-112576.0],15.6,10.7,1.32
3,Indonesia,36633,[33763.5-39746.4],27.0,24.4,3.55
4,Brazil,17743,[16977.0-18543.6],16.4,12.7,1.99
...,...,...,...,...,...,...
181,French Polynesia,16,[8.6-29.7],11.5,9.0,1.14
182,Iceland,16,[9.6-26.6],9.4,8.3,0.74
183,Sao Tome and Principe,13,[12.5-13.6],11.9,16.0,4.67
184,Malta,13,[6.6-25.8],5.9,3.7,0.53


In [3]:
# taking a look
df_incid_countries.describe()

,Number,Crude Rate,ASR (World),Cum. risk
count,186.000000,186.000000,186.000000,186.000000
mean,6486.403226,16.660215,18.050538,2.709086
std,45804.148287,9.566977,14.215117,2.199703
min,10.000000,1.400000,2.200000,0.330000
25%,179.750000,10.200000,8.025000,1.190000
50%,574.000000,15.100000,14.600000,2.075000
75%,2046.500000,21.575000,21.750000,3.557500
max,604127.000000,57.800000,84.500000,11.580000


In [4]:
# looking at column types
df_incid_countries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 186 entries, 0 to 185
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Population            186 non-null    object 
 1   Number                186 non-null    int64  
 2   Uncertainty interval  186 non-null    object 
 3   Crude Rate            186 non-null    float64
 4   ASR (World)           186 non-null    float64
 5   Cum. risk             186 non-null    float64
dtypes: float64(3), int64(1), object(2)
memory usage: 8.8+ KB


In [5]:
# load second dataset with CC mortality rates
df_mort_countries = pd.read_csv('cc_mortality_countries_2020_iarc.csv', encoding='latin-1', index_col=False)


In [6]:
df_incid_countries.columns

Index(['Population', 'Number', 'Uncertainty interval', 'Crude Rate',
       'ASR (World)', 'Cum. risk'],
      dtype='object')

**I don't want to include the crude rates on the dashboard as it's difficult to compare countries with crude rates as they all have different populations. Instead I will keep ASR (World) (which is the age-standarised incidence rate) and I will remove the other columns.**

In [7]:
#removing extra columns
df_incid_countries.drop(['Number', 'Uncertainty interval', 'Crude Rate', 'Cum. risk'], axis=1, inplace=True)
df_incid_countries.columns

Index(['Population', 'ASR (World)'], dtype='object')

In [8]:
#renaming columns
df_incid_countries.columns = ['Country', 'Incidence']
df_incid_countries.head()

,Country,Incidence
0,World,13.3
1,India,18.0
2,China,10.7
3,Indonesia,24.4
4,Brazil,12.7


In [9]:
df_incid_countries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 186 entries, 0 to 185
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Country    186 non-null    object 
 1   Incidence  186 non-null    float64
dtypes: float64(1), object(1)
memory usage: 3.0+ KB


In [10]:
#repeating process for mortality dataset
df_mort_countries.drop(['Number', 'Uncertainty interval', 'Crude Rate', 'Cum. risk'], axis=1, inplace=True)
df_mort_countries.columns = ['Country', 'Mortality']

In [11]:
df_incid_countries.reset_index(drop=True, inplace=True)
df_mort_countries.reset_index(drop=True, inplace=True)

In [12]:
df_mort_countries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 186 entries, 0 to 185
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Country    186 non-null    object 
 1   Mortality  186 non-null    float64
dtypes: float64(1), object(1)
memory usage: 3.0+ KB


In [13]:
df_incid_countries.sort_values(by=['Country'], inplace=True)

In [14]:
df_incid_countries.head(10)

,Country,Incidence
67,Afghanistan,10.4
145,Albania,6.6
55,Algeria,7.9
36,Angola,37.6
25,Argentina,16.7
139,Armenia,7.8
77,Australia,5.6
106,Austria,5.3
102,Azerbaijan,6.7
168,Bahamas,14.9


In [15]:
df_mort_countries.sort_values(by=['Country'], inplace=True)
df_mort_countries.head()

,Country,Mortality
60,Afghanistan,7.6
143,Albania,3.3
54,Algeria,4.5
33,Angola,25.0
23,Argentina,8.7


In [16]:
df_incid_countries.drop(0, axis=0, inplace=True)
df_incid_countries.info()

<class 'pandas.core.frame.DataFrame'>
Index: 185 entries, 67 to 39
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Country    185 non-null    object 
 1   Incidence  185 non-null    float64
dtypes: float64(1), object(1)
memory usage: 4.3+ KB


In [17]:
df_mort_countries.drop(0, axis=0, inplace=True)
df_mort_countries.info()

<class 'pandas.core.frame.DataFrame'>
Index: 185 entries, 60 to 32
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Country    185 non-null    object 
 1   Mortality  185 non-null    float64
dtypes: float64(1), object(1)
memory usage: 4.3+ KB


In [18]:
#merging datasets
df_countries = pd.merge(df_incid_countries, df_mort_countries, on="Country")
df_countries.head()

,Country,Incidence,Mortality
0,Afghanistan,10.4,7.6
1,Albania,6.6,3.3
2,Algeria,7.9,4.5
3,Angola,37.6,25.0
4,Argentina,16.7,8.7


In [19]:
df_countries['Country'].values

array(['Afghanistan', 'Albania', 'Algeria', 'Angola', 'Argentina',
       'Armenia', 'Australia', 'Austria', 'Azerbaijan', 'Bahamas',
       'Bahrain', 'Bangladesh', 'Barbados', 'Belarus', 'Belgium',
       'Belize', 'Benin', 'Bhutan', 'Bolivia, Plurinational State of',
       'Bosnia and Herzegovina', 'Botswana', 'Brazil',
       'Brunei Darussalam', 'Bulgaria', 'Burkina Faso', 'Burundi',
       'Cabo Verde', 'Cambodia', 'Cameroon', 'Canada',
       'Central African Republic', 'Chad', 'Chile', 'China', 'Colombia',
       'Comoros', 'Congo, Democratic Republic of', 'Congo, Republic of',
       'Costa Rica', 'Croatia', 'Cuba', 'Cyprus', 'Czechia',
       "Côte d'Ivoire", 'Denmark', 'Djibouti', 'Dominican Republic',
       'Ecuador', 'Egypt', 'El Salvador', 'Equatorial Guinea', 'Eritrea',
       'Estonia', 'Eswatini', 'Ethiopia', 'Fiji', 'Finland', 'France',
       'France, Guadeloupe', 'France, La Réunion', 'France, Martinique',
       'France, New Caledonia', 'French Guiana', 'French P

In [20]:
# removing countries which we don't have country codes for
df_countries = df_countries[df_countries.Country != 'France, Guadeloupe']
df_countries = df_countries[df_countries.Country != 'France, La Réunion']
df_countries = df_countries[df_countries.Country != 'France, Martinique']
df_countries = df_countries[df_countries.Country != 'France, New Caledonia']
df_countries = df_countries[df_countries.Country != 'Gaza Strip and West Bank']
df_countries = df_countries[df_countries.Country != 'Timor-Leste']

In [21]:
# changing the names of countries to get their country code
df_countries.replace("Korea, Democratic Republic of", "Korea, Republic of", inplace=True)
df_countries.replace("The Netherlands", "Netherlands", inplace=True)
df_countries.replace("The Republic of the Gambia", "Gambia", inplace=True)

In [22]:
# pip install pycountry

from pycountry_convert import country_alpha2_to_continent_code, country_name_to_country_alpha2
def get_continent(col):
    try:
        cn_a2_code =  country_name_to_country_alpha2(col)
    except:
        cn_a2_code = 'Unknown' 
    try:
        cn_continent = country_alpha2_to_continent_code(cn_a2_code)
    except:
        cn_continent = 'Unknown' 
    return (cn_a2_code, cn_continent)

In [23]:
list(pycountry.countries)

[Country(alpha_2='AW', alpha_3='ABW', flag='🇦🇼', name='Aruba', numeric='533'),
 Country(alpha_2='AF', alpha_3='AFG', flag='🇦🇫', name='Afghanistan', numeric='004', official_name='Islamic Republic of Afghanistan'),
 Country(alpha_2='AO', alpha_3='AGO', flag='🇦🇴', name='Angola', numeric='024', official_name='Republic of Angola'),
 Country(alpha_2='AI', alpha_3='AIA', flag='🇦🇮', name='Anguilla', numeric='660'),
 Country(alpha_2='AX', alpha_3='ALA', flag='🇦🇽', name='Åland Islands', numeric='248'),
 Country(alpha_2='AL', alpha_3='ALB', flag='🇦🇱', name='Albania', numeric='008', official_name='Republic of Albania'),
 Country(alpha_2='AD', alpha_3='AND', flag='🇦🇩', name='Andorra', numeric='020', official_name='Principality of Andorra'),
 Country(alpha_2='AE', alpha_3='ARE', flag='🇦🇪', name='United Arab Emirates', numeric='784'),
 Country(alpha_2='AR', alpha_3='ARG', flag='🇦🇷', name='Argentina', numeric='032', official_name='Argentine Republic'),
 Country(alpha_2='AM', alpha_3='ARM', flag='🇦🇲', 

In [24]:
import pycountry_convert as pc

def country_to_continent(country_name):
    country_alpha3 = pc.country_name_to_country_alpha3(country_name)
    country_alpha2 = pc.country_name_to_country_alpha2(country_name)
    country_continent_code = pc.country_alpha2_to_continent_code(country_alpha2)
    country_continent_name = pc.convert_continent_code_to_continent_name(country_continent_code)
    return country_alpha2, country_alpha3, country_continent_name

# Example
country_name = 'Germany'
print(country_to_continent(country_name))

('DE', 'DEU', 'Europe')


In [25]:
# adding columns
df_countries['Code2'] = [country_to_continent(country)[0] for country in df_countries['Country']]
df_countries['Code3'] = [country_to_continent(country)[1] for country in df_countries['Country']]
df_countries['Continent'] = [country_to_continent(country)[2] for country in df_countries['Country']]

In [26]:
df_countries

,Country,Incidence,Mortality,Code2,Code3,Continent
0,Afghanistan,10.4,7.6,AF,AFG,Asia
1,Albania,6.6,3.3,AL,ALB,Europe
2,Algeria,7.9,4.5,DZ,DZA,Africa
3,Angola,37.6,25.0,AO,AGO,Africa
4,Argentina,16.7,8.7,AR,ARG,South America
...,...,...,...,...,...,...
180,"Venezuela, Bolivarian Republic of",22.2,12.5,VE,VEN,South America
181,Viet Nam,6.6,3.4,VN,VNM,Asia
182,Yemen,2.5,1.8,YE,YEM,Asia
183,Zambia,65.5,43.4,ZM,ZMB,Africa


In [27]:
countries = list(df_countries['Country']) #checking which countries we have data for
print(len(countries))
print(countries)

179
['Afghanistan', 'Albania', 'Algeria', 'Angola', 'Argentina', 'Armenia', 'Australia', 'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain', 'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin', 'Bhutan', 'Bolivia, Plurinational State of', 'Bosnia and Herzegovina', 'Botswana', 'Brazil', 'Brunei Darussalam', 'Bulgaria', 'Burkina Faso', 'Burundi', 'Cabo Verde', 'Cambodia', 'Cameroon', 'Canada', 'Central African Republic', 'Chad', 'Chile', 'China', 'Colombia', 'Comoros', 'Congo, Democratic Republic of', 'Congo, Republic of', 'Costa Rica', 'Croatia', 'Cuba', 'Cyprus', 'Czechia', "Côte d'Ivoire", 'Denmark', 'Djibouti', 'Dominican Republic', 'Ecuador', 'Egypt', 'El Salvador', 'Equatorial Guinea', 'Eritrea', 'Estonia', 'Eswatini', 'Ethiopia', 'Fiji', 'Finland', 'France', 'French Guiana', 'French Polynesia', 'Gabon', 'Georgia', 'Germany', 'Ghana', 'Greece', 'Guam', 'Guatemala', 'Guinea', 'Guinea-Bissau', 'Guyana', 'Haiti', 'Honduras', 'Hungary', 'Iceland', 'India', 'Indonesia', 'Iran, 

In [28]:
# loading third dataset with HDI and components
df_HDI = pd.read_csv('2020_HDI_and_components.csv', index_col=0)
df_HDI

,Country,Human Development Index (HDI),Life expectancy at birth,Expected years of schooling,Mean years of schooling,Gross national income (GNI) per capita,GNI per capita rank minus HDI rank
HDI rank,,,,,,,
1,Norway,0.957,82.4,18.1,12.9,"66,494",7
2,Ireland,0.955,82.3,18.7,12.7,"68,371",4
2,Switzerland,0.955,83.8,16.3,13.4,"69,394",3
4,"Hong Kong, China (SAR)",0.949,84.9,16.9,12.3,"62,985",7
4,Iceland,0.949,83.0,19.1,12.8,"54,682",14
...,...,...,...,...,...,...,...
185,Burundi,0.433,61.6,11.1,3.3,754,4
185,South Sudan,0.433,57.9,5.3,4.8,"2,003",-10
187,Chad,0.398,54.2,7.3,2.5,"1,555",-5


In [29]:
df_HDI.info()

<class 'pandas.core.frame.DataFrame'>
Index: 189 entries, 1 to 189
Data columns (total 7 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   Country                                 189 non-null    object 
 1   Human Development Index (HDI)           189 non-null    float64
 2   Life expectancy at birth                189 non-null    float64
 3   Expected years of schooling             189 non-null    float64
 4   Mean years of schooling                 189 non-null    float64
 5   Gross national income (GNI) per capita  189 non-null    object 
 6   GNI per capita rank minus HDI rank      189 non-null    int64  
dtypes: float64(4), int64(1), object(2)
memory usage: 11.8+ KB


In [30]:
df_HDI.dropna(inplace=True, axis=1)

In [31]:
df_HDI

,Country,Human Development Index (HDI),Life expectancy at birth,Expected years of schooling,Mean years of schooling,Gross national income (GNI) per capita,GNI per capita rank minus HDI rank
HDI rank,,,,,,,
1,Norway,0.957,82.4,18.1,12.9,"66,494",7
2,Ireland,0.955,82.3,18.7,12.7,"68,371",4
2,Switzerland,0.955,83.8,16.3,13.4,"69,394",3
4,"Hong Kong, China (SAR)",0.949,84.9,16.9,12.3,"62,985",7
4,Iceland,0.949,83.0,19.1,12.8,"54,682",14
...,...,...,...,...,...,...,...
185,Burundi,0.433,61.6,11.1,3.3,754,4
185,South Sudan,0.433,57.9,5.3,4.8,"2,003",-10
187,Chad,0.398,54.2,7.3,2.5,"1,555",-5


In [32]:
HDI_countries = list(df_HDI['Country']) #checking which countries we have data for
print(len(HDI_countries))
print(HDI_countries)

189
['Norway', 'Ireland', 'Switzerland', 'Hong Kong, China (SAR)', 'Iceland', 'Germany', 'Sweden', 'Australia', 'Netherlands', 'Denmark', 'Finland', 'Singapore', 'United Kingdom', 'Belgium', 'New Zealand', 'Canada', 'United States', 'Austria', 'Israel', 'Japan', 'Liechtenstein', 'Slovenia', 'Korea (Republic of)', 'Luxembourg', 'Spain', 'France', 'Czechia', 'Malta', 'Estonia', 'Italy', 'United Arab Emirates', 'Greece', 'Cyprus', 'Lithuania', 'Poland', 'Andorra', 'Latvia', 'Portugal', 'Slovakia', 'Hungary', 'Saudi Arabia', 'Bahrain', 'Chile', 'Croatia', 'Qatar', 'Argentina', 'Brunei Darussalam', 'Montenegro', 'Romania', 'Palau', 'Kazakhstan', 'Russian Federation', 'Belarus', 'Turkey', 'Uruguay', 'Bulgaria', 'Panama', 'Bahamas', 'Barbados', 'Oman', 'Georgia', 'Costa Rica', 'Malaysia', 'Kuwait', 'Serbia', 'Mauritius', 'Seychelles', 'Trinidad and Tobago', 'Albania', 'Cuba', 'Iran (Islamic Republic of)', 'Sri Lanka', 'Bosnia and Herzegovina', 'Grenada', 'Mexico', 'Saint Kitts and Nevis', 'Uk

In [33]:
# finding countries which are not found in main dataset
not_in_countries_df = df_HDI[~df_HDI['Country'].isin(countries)]
not_in_countries_df

,Country,Human Development Index (HDI),Life expectancy at birth,Expected years of schooling,Mean years of schooling,Gross national income (GNI) per capita,GNI per capita rank minus HDI rank
HDI rank,,,,,,,
4,"Hong Kong, China (SAR)",0.949,84.9,16.9,12.3,"62,985",7
17,United States,0.926,78.9,16.3,13.4,"63,826",-7
19,Liechtenstein,0.919,80.7,14.9,12.5,"131,032",-18
23,Korea (Republic of),0.916,83.0,16.5,12.2,"43,044",4
36,Andorra,0.868,81.9,13.3,10.5,"56,000",-20
50,Palau,0.826,73.9,15.8,12.5,"19,317",15
67,Seychelles,0.796,73.4,14.1,10.0,"26,903",-15
70,Iran (Islamic Republic of),0.783,76.7,14.8,10.3,"12,447",26
74,Grenada,0.779,72.4,16.9,9.0,"15,641",3


In [34]:
# changing names to conform with main dataset
df_HDI.replace("United States", "United States of America", inplace=True)
df_HDI.replace("Korea (Republic of)", "Korea, Republic of", inplace=True)
df_HDI.replace("Iran (Islamic Republic of)", "Iran, Islamic Republic of", inplace=True)
df_HDI.replace("Moldova (Republic of)", "Republic of Moldova", inplace=True)
df_HDI.replace("Bolivia (Plurinational State of)", "Bolivia, Plurinational State of", inplace=True)
df_HDI.replace("Eswatini (Kingdom of)", "Eswatini", inplace=True)
df_HDI.replace("Tanzania (United Republic of)", "Tanzania, United Republic of", inplace=True)
df_HDI.replace("Venezuela (Bolivarian Republic of)", "Venezuela, Bolivarian Republic of", inplace=True)
df_HDI.replace("Congo (Democratic Republic of the)", "Congo, Democratic Republic of", inplace=True)
df_HDI.replace("Congo", "Congo, Republic of", inplace=True)

In [35]:
not_in_countries_df = df_HDI[~df_HDI['Country'].isin(countries)]
not_in_countries_df

,Country,Human Development Index (HDI),Life expectancy at birth,Expected years of schooling,Mean years of schooling,Gross national income (GNI) per capita,GNI per capita rank minus HDI rank
HDI rank,,,,,,,
4,"Hong Kong, China (SAR)",0.949,84.9,16.9,12.3,"62,985",7
19,Liechtenstein,0.919,80.7,14.9,12.5,"131,032",-18
36,Andorra,0.868,81.9,13.3,10.5,"56,000",-20
50,Palau,0.826,73.9,15.8,12.5,"19,317",15
67,Seychelles,0.796,73.4,14.1,10.0,"26,903",-15
74,Grenada,0.779,72.4,16.9,9.0,"15,641",3
74,Saint Kitts and Nevis,0.779,74.8,13.8,8.7,"25,038",-17
78,Antigua and Barbuda,0.778,77.0,12.8,9.3,"20,895",-15
94,Dominica,0.742,78.2,13.0,8.1,"11,884",7


In [36]:
# only keeping the countries which are in the main CC dataset
df_HDI_common_countries = df_HDI[df_HDI['Country'].isin(countries)]
df_HDI_common_countries

,Country,Human Development Index (HDI),Life expectancy at birth,Expected years of schooling,Mean years of schooling,Gross national income (GNI) per capita,GNI per capita rank minus HDI rank
HDI rank,,,,,,,
1,Norway,0.957,82.4,18.1,12.9,"66,494",7
2,Ireland,0.955,82.3,18.7,12.7,"68,371",4
2,Switzerland,0.955,83.8,16.3,13.4,"69,394",3
4,Iceland,0.949,83.0,19.1,12.8,"54,682",14
6,Germany,0.947,81.3,17.0,14.2,"55,314",11
...,...,...,...,...,...,...,...
185,Burundi,0.433,61.6,11.1,3.3,754,4
185,South Sudan,0.433,57.9,5.3,4.8,"2,003",-10
187,Chad,0.398,54.2,7.3,2.5,"1,555",-5


In [37]:
HDI_common_countries = list(df_HDI_common_countries['Country'])
df_countries[~df_countries['Country'].isin(HDI_common_countries)]

,Country,Incidence,Mortality,Code2,Code3,Continent
62,French Guiana,12.6,6.5,GF,GUF,South America
63,French Polynesia,9.0,4.5,PF,PYF,Oceania
70,Guam,18.4,8.0,GU,GUM,Oceania
134,Puerto Rico,8.0,2.8,PR,PRI,North America
151,Somalia,25.1,20.2,SO,SOM,Africa


In [38]:
df_HDI.sort_values(by=['Country'], inplace=True)
HDI_countries = list(df_HDI['Country'])
print(HDI_countries)

['Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola', 'Antigua and Barbuda', 'Argentina', 'Armenia', 'Australia', 'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain', 'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin', 'Bhutan', 'Bolivia, Plurinational State of', 'Bosnia and Herzegovina', 'Botswana', 'Brazil', 'Brunei Darussalam', 'Bulgaria', 'Burkina Faso', 'Burundi', 'Cabo Verde', 'Cambodia', 'Cameroon', 'Canada', 'Central African Republic', 'Chad', 'Chile', 'China', 'Colombia', 'Comoros', 'Congo, Democratic Republic of', 'Congo, Republic of', 'Costa Rica', 'Croatia', 'Cuba', 'Cyprus', 'Czechia', "Côte d'Ivoire", 'Denmark', 'Djibouti', 'Dominica', 'Dominican Republic', 'Ecuador', 'Egypt', 'El Salvador', 'Equatorial Guinea', 'Eritrea', 'Estonia', 'Eswatini', 'Ethiopia', 'Fiji', 'Finland', 'France', 'Gabon', 'Gambia', 'Georgia', 'Germany', 'Ghana', 'Greece', 'Grenada', 'Guatemala', 'Guinea', 'Guinea-Bissau', 'Guyana', 'Haiti', 'Honduras', 'Hong Kong, China (SAR)', 'Hungar

In [39]:
df_countries = df_countries[df_countries['Country'].isin(HDI_common_countries)]
df_countries.info()

<class 'pandas.core.frame.DataFrame'>
Index: 174 entries, 0 to 184
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Country    174 non-null    object 
 1   Incidence  174 non-null    float64
 2   Mortality  174 non-null    float64
 3   Code2      174 non-null    object 
 4   Code3      174 non-null    object 
 5   Continent  174 non-null    object 
dtypes: float64(2), object(4)
memory usage: 9.5+ KB


In [40]:
df_HDI_common_countries.info()

<class 'pandas.core.frame.DataFrame'>
Index: 173 entries, 1 to 189
Data columns (total 7 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   Country                                 173 non-null    object 
 1   Human Development Index (HDI)           173 non-null    float64
 2   Life expectancy at birth                173 non-null    float64
 3   Expected years of schooling             173 non-null    float64
 4   Mean years of schooling                 173 non-null    float64
 5   Gross national income (GNI) per capita  173 non-null    object 
 6   GNI per capita rank minus HDI rank      173 non-null    int64  
dtypes: float64(4), int64(1), object(2)
memory usage: 10.8+ KB


In [41]:
df_HDI_common_countries.sort_values(by=['Country'], inplace=True)
df_HDI_common_countries.head()

/tmp/ipykernel_2111/2840971854.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Country,Human Development Index (HDI),Life expectancy at birth,Expected years of schooling,Mean years of schooling,Gross national income (GNI) per capita,GNI per capita rank minus HDI rank
HDI rank,,,,,,,
169,Afghanistan,0.511,64.8,10.2,3.9,"2,229",0
69,Albania,0.795,78.6,14.7,10.1,"13,998",18
91,Algeria,0.748,76.9,14.6,8.0,"11,174",13
148,Angola,0.581,61.2,11.8,5.2,"6,104",-17
46,Argentina,0.845,76.7,17.7,10.9,"21,190",16


In [42]:
# joining datasets
df_countries2 = pd.merge(df_countries, df_HDI_common_countries, on="Country")
df_countries2.head(50)

,Country,Incidence,Mortality,Code2,Code3,Continent,Human Development Index (HDI),Life expectancy at birth,Expected years of schooling,Mean years of schooling,Gross national income (GNI) per capita,GNI per capita rank minus HDI rank
0,Afghanistan,10.4,7.6,AF,AFG,Asia,0.511,64.8,10.2,3.9,"2,229",0
1,Albania,6.6,3.3,AL,ALB,Europe,0.795,78.6,14.7,10.1,"13,998",18
2,Algeria,7.9,4.5,DZ,DZA,Africa,0.748,76.9,14.6,8.0,"11,174",13
3,Angola,37.6,25.0,AO,AGO,Africa,0.581,61.2,11.8,5.2,"6,104",-17
4,Argentina,16.7,8.7,AR,ARG,South America,0.845,76.7,17.7,10.9,"21,190",16
5,Armenia,7.8,4.6,AM,ARM,Asia,0.776,75.1,13.1,11.3,"13,894",9
6,Australia,5.6,1.5,AU,AUS,Oceania,0.944,83.4,22.0,12.7,"48,085",15
7,Austria,5.3,1.8,AT,AUT,Europe,0.922,81.5,16.1,12.5,"56,197",-3
8,Azerbaijan,6.7,4.0,AZ,AZE,Asia,0.756,73.0,12.9,10.6,"13,784",3
9,Bahamas,14.9,10.6,BS,BHS,North America,0.814,73.9,12.9,11.4,"33,747",-17


In [43]:
# loading fourth dataset with is a list of countries with have national HPV vaccination programs as of Aug 2021
df_hpv = pd.read_csv('hpv_vaccine_countries.csv')
df_hpv

,Country
0,Argentina
1,Armenia
2,Australia
3,Austria
4,Bahamas
...,...
105,United States of America
106,Uruguay
107,Uzbekistan
108,Zambia


In [44]:
df_hpv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110 entries, 0 to 109
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Country  110 non-null    object
dtypes: object(1)
memory usage: 1012.0+ bytes


In [45]:
hpv_vaccine_countries = list(df_hpv['Country'])

In [46]:
vaccine_program = []
for country in df_countries2['Country']:
    if country in hpv_vaccine_countries:
        vaccine_program.append('Yes')
    else:
        vaccine_program.append('No')

vaccine_program

['No',
 'No',
 'No',
 'No',
 'No',
 'No',
 'Yes',
 'Yes',
 'No',
 'No',
 'No',
 'No',
 'Yes',
 'No',
 'No',
 'Yes',
 'No',
 'Yes',
 'Yes',
 'No',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'No',
 'No',
 'No',
 'No',
 'Yes',
 'Yes',
 'No',
 'No',
 'No',
 'No',
 'Yes',
 'No',
 'No',
 'No',
 'Yes',
 'Yes',
 'No',
 'Yes',
 'Yes',
 'No',
 'No',
 'No',
 'Yes',
 'No',
 'No',
 'Yes',
 'No',
 'No',
 'Yes',
 'No',
 'No',
 'Yes',
 'No',
 'Yes',
 'No',
 'Yes',
 'Yes',
 'No',
 'Yes',
 'Yes',
 'No',
 'No',
 'Yes',
 'No',
 'Yes',
 'No',
 'No',
 'No',
 'No',
 'No',
 'No',
 'No',
 'Yes',
 'Yes',
 'Yes',
 'No',
 'No',
 'No',
 'Yes',
 'No',
 'No',
 'No',
 'No',
 'Yes',
 'Yes',
 'No',
 'No',
 'Yes',
 'Yes',
 'Yes',
 'No',
 'No',
 'Yes',
 'Yes',
 'Yes',
 'No',
 'Yes',
 'Yes',
 'No',
 'Yes',
 'No',
 'No',
 'Yes',
 'No',
 'No',
 'No',
 'No',
 'Yes',
 'No',
 'No',
 'No',
 'No',
 'Yes',
 'No',
 'No',
 'Yes',
 'No',
 'No',
 'Yes',
 'Yes',
 'No',
 'Yes',
 'Yes',
 'No',
 'No',
 'No',
 'No',
 'No',
 'No',
 'No',
 'No',
 'Ye

In [47]:
# creating new column in dataset which specifies if country has a HPV vaccination program
df_countries2['Vaccine_Program'] = vaccine_program
df_countries2.head(20)

,Country,Incidence,Mortality,Code2,Code3,Continent,Human Development Index (HDI),Life expectancy at birth,Expected years of schooling,Mean years of schooling,Gross national income (GNI) per capita,GNI per capita rank minus HDI rank,Vaccine_Program
0,Afghanistan,10.4,7.6,AF,AFG,Asia,0.511,64.8,10.2,3.9,"2,229",0,No
1,Albania,6.6,3.3,AL,ALB,Europe,0.795,78.6,14.7,10.1,"13,998",18,No
2,Algeria,7.9,4.5,DZ,DZA,Africa,0.748,76.9,14.6,8.0,"11,174",13,No
3,Angola,37.6,25.0,AO,AGO,Africa,0.581,61.2,11.8,5.2,"6,104",-17,No
4,Argentina,16.7,8.7,AR,ARG,South America,0.845,76.7,17.7,10.9,"21,190",16,No
5,Armenia,7.8,4.6,AM,ARM,Asia,0.776,75.1,13.1,11.3,"13,894",9,No
6,Australia,5.6,1.5,AU,AUS,Oceania,0.944,83.4,22.0,12.7,"48,085",15,Yes
7,Austria,5.3,1.8,AT,AUT,Europe,0.922,81.5,16.1,12.5,"56,197",-3,Yes
8,Azerbaijan,6.7,4.0,AZ,AZE,Asia,0.756,73.0,12.9,10.6,"13,784",3,No
9,Bahamas,14.9,10.6,BS,BHS,North America,0.814,73.9,12.9,11.4,"33,747",-17,No


In [48]:
# loading one more dataset with sub-regions
df_subregions = pd.read_csv('world_data.csv')
df_subregions

,name,alpha-2,alpha-3,country-code,iso_3166-2,region,sub-region,intermediate-region,region-code,sub-region-code,intermediate-region-code
0,Afghanistan,AF,AFG,4,ISO 3166-2:AF,Asia,Southern Asia,NaN,142.0,34.0,NaN
1,Åland Islands,AX,ALA,248,ISO 3166-2:AX,Europe,Northern Europe,NaN,150.0,154.0,NaN
2,Albania,AL,ALB,8,ISO 3166-2:AL,Europe,Southern Europe,NaN,150.0,39.0,NaN
3,Algeria,DZ,DZA,12,ISO 3166-2:DZ,Africa,Northern Africa,NaN,2.0,15.0,NaN
4,American Samoa,AS,ASM,16,ISO 3166-2:AS,Oceania,Polynesia,NaN,9.0,61.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
244,Wallis and Futuna,WF,WLF,876,ISO 3166-2:WF,Oceania,Polynesia,NaN,9.0,61.0,NaN
245,Western Sahara,EH,ESH,732,ISO 3166-2:EH,Africa,Northern Africa,NaN,2.0,15.0,NaN
246,Yemen,YE,YEM,887,ISO 3166-2:YE,Asia,Western Asia,NaN,142.0,145.0,NaN
247,Zambia,ZM,ZMB,894,ISO 3166-2:ZM,Africa,Sub-Saharan Africa,Eastern Africa,2.0,202.0,14.0


In [49]:
df_subregions.columns

Index(['name', 'alpha-2', 'alpha-3', 'country-code', 'iso_3166-2', 'region',
       'sub-region', 'intermediate-region', 'region-code', 'sub-region-code',
       'intermediate-region-code'],
      dtype='object')

In [50]:
df_subregions.drop(['name', 'alpha-2', 'country-code', 'iso_3166-2', 'region',
       'intermediate-region', 'region-code', 'sub-region-code',
       'intermediate-region-code'], inplace=True, axis=1)

In [51]:
df_subregions.columns = ['Code3', 'Sub-Region']
df_subregions.head()

,Code3,Sub-Region
0,AFG,Southern Asia
1,ALA,Northern Europe
2,ALB,Southern Europe
3,DZA,Northern Africa
4,ASM,Polynesia


In [52]:
# merging datasets
df_countries3 = pd.merge(df_countries2, df_subregions, on="Code3")
df_countries3.head()

,Country,Incidence,Mortality,Code2,Code3,Continent,Human Development Index (HDI),Life expectancy at birth,Expected years of schooling,Mean years of schooling,Gross national income (GNI) per capita,GNI per capita rank minus HDI rank,Vaccine_Program,Sub-Region
0,Afghanistan,10.4,7.6,AF,AFG,Asia,0.511,64.8,10.2,3.9,"2,229",0,No,Southern Asia
1,Albania,6.6,3.3,AL,ALB,Europe,0.795,78.6,14.7,10.1,"13,998",18,No,Southern Europe
2,Algeria,7.9,4.5,DZ,DZA,Africa,0.748,76.9,14.6,8.0,"11,174",13,No,Northern Africa
3,Angola,37.6,25.0,AO,AGO,Africa,0.581,61.2,11.8,5.2,"6,104",-17,No,Sub-Saharan Africa
4,Argentina,16.7,8.7,AR,ARG,South America,0.845,76.7,17.7,10.9,"21,190",16,No,Latin America and the Caribbean


In [53]:
df_countries3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 174 entries, 0 to 173
Data columns (total 14 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   Country                                 174 non-null    object 
 1   Incidence                               174 non-null    float64
 2   Mortality                               174 non-null    float64
 3   Code2                                   174 non-null    object 
 4   Code3                                   174 non-null    object 
 5   Continent                               174 non-null    object 
 6   Human Development Index (HDI)           174 non-null    float64
 7   Life expectancy at birth                174 non-null    float64
 8   Expected years of schooling             174 non-null    float64
 9   Mean years of schooling                 174 non-null    float64
 10  Gross national income (GNI) per capita  174 non-null    object

In [54]:
# removing commas in column so it can be converted to a float
df_countries3['Gross national income (GNI) per capita']=df_countries3['Gross national income (GNI) per capita'].str.replace(',','')
df_countries3 = df_countries3.astype({"Gross national income (GNI) per capita": float})
df_countries3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 174 entries, 0 to 173
Data columns (total 14 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   Country                                 174 non-null    object 
 1   Incidence                               174 non-null    float64
 2   Mortality                               174 non-null    float64
 3   Code2                                   174 non-null    object 
 4   Code3                                   174 non-null    object 
 5   Continent                               174 non-null    object 
 6   Human Development Index (HDI)           174 non-null    float64
 7   Life expectancy at birth                174 non-null    float64
 8   Expected years of schooling             174 non-null    float64
 9   Mean years of schooling                 174 non-null    float64
 10  Gross national income (GNI) per capita  174 non-null    float6

In [55]:
df_countries3.columns

Index(['Country', 'Incidence', 'Mortality', 'Code2', 'Code3', 'Continent',
       'Human Development Index (HDI) ', 'Life expectancy at birth',
       'Expected years of schooling', 'Mean years of schooling',
       'Gross national income (GNI) per capita',
       'GNI per capita rank minus HDI rank', 'Vaccine_Program', 'Sub-Region'],
      dtype='object')

In [56]:
# renaming columns for dashboard

df_countries3 = df_countries3.rename(columns={'Human Development Index (HDI) ': "Human Development Index (HDI)",
                                              'Life expectancy at birth':'Life Expectancy at Birth',
                                              'Expected years of schooling':'Expected Years of Schooling',
                                              'Mean years of schooling':'Mean Years of Schooling',
                                              'Gross national income (GNI) per capita':'Gross National Income (GNI) per Capita',
                                              'Vaccine_Program':'National HPV Vaccination Program'
                                            })

In [57]:
df_countries3.head()

,Country,Incidence,Mortality,Code2,Code3,Continent,Human Development Index (HDI),Life Expectancy at Birth,Expected Years of Schooling,Mean Years of Schooling,Gross National Income (GNI) per Capita,GNI per capita rank minus HDI rank,National HPV Vaccination Program,Sub-Region
0,Afghanistan,10.4,7.6,AF,AFG,Asia,0.511,64.8,10.2,3.9,2229.0,0,No,Southern Asia
1,Albania,6.6,3.3,AL,ALB,Europe,0.795,78.6,14.7,10.1,13998.0,18,No,Southern Europe
2,Algeria,7.9,4.5,DZ,DZA,Africa,0.748,76.9,14.6,8.0,11174.0,13,No,Northern Africa
3,Angola,37.6,25.0,AO,AGO,Africa,0.581,61.2,11.8,5.2,6104.0,-17,No,Sub-Saharan Africa
4,Argentina,16.7,8.7,AR,ARG,South America,0.845,76.7,17.7,10.9,21190.0,16,No,Latin America and the Caribbean


### Some Exploratory Data Analysis

In [58]:
fig = px.choropleth(df_countries, locations='Code3', color='Incidence',
                           color_continuous_scale="bluered",
                          )
fig.show()

In [59]:
data = [go.Scatter(
    x = df_countries['Incidence'],
    y = df_countries['Mortality'],
    mode = 'markers',
    marker = dict(  #change the marker style
        size = 12,
        color = 'rgb(51,204,153)',
        symbol = 'pentagon',
        line = dict(
            width = 2,
        )
    )
)]

layout = go.Layout(
    title = 'Incidence Vs. Mortality', # Graph title
    xaxis = dict(title = 'Incidence'), # x-axis label
    yaxis = dict(title = 'Mortality'), # y-axis label
    hovermode = 'closest' # handles multiple points landing on the same vertical 
)
fig = go.Figure(data=data, layout=layout)
#pyo.plot(fig, filename='scatter.html')
fig.show()

In [60]:
# grouped bar chart showing two traces

trace1 = go.Bar(
    x = df_countries3['Sub-Region'],
    y = df_countries3['Incidence'],
    name = 'Incidence',
    marker=dict(color='#FFD700') 
)

trace2 = go.Bar(
    x = df_countries3['Sub-Region'],
    y = df_countries3['Mortality'],
    name = 'Mortality',
    marker=dict(color='#9EA0A1') #setting marker color to silver
)


data= trace1
layout= go.Layout(
    title='Incidence and Mortality by Sub-Region'
)

fig = go.Figure(data=data, layout=layout)
#pyo.plot(fig, filename='contintent_bar.html')
fig.show()

In [61]:
fig = px.histogram(df_countries3, x="Sub-Region", y="Incidence",
             barmode='group',
             histfunc='avg',
             height=400,
            hover_name="Sub-Region",
            text_auto='.1f',
            hover_data = {"Incidence": True},
            labels={"Incidence": "Average Rate"},
            title="Rates by Sub-Region")
fig.update_xaxes(categoryorder='total ascending')
#fig.update_traces(texttemplate='%{text:.2s}', textposition="auto")
fig.show()

In [62]:
data = px.scatter(
    x = df_countries3['Human Development Index (HDI)'],
    y = df_countries3['Mortality'], trendline='ols')



layout = go.Layout(
    title = 'HDI Vs. Mortality', # Graph title
    xaxis = dict(title = 'HDI'), # x-axis label
    yaxis = dict(title = 'Mortality'), # y-axis label
    hovermode = 'closest' # handles multiple points landing on the same vertical 
)
fig = go.Figure(data=data, layout=layout)

# make the regression line stand out
fig.data[1].line.color = 'red'
#pyo.plot(fig, filename='scatter.html')
fig.show()

In [63]:
data = [go.Scatter(
    x = df_countries3['Human Development Index (HDI)'],
    y = df_countries3['Incidence'], 
    mode = 'markers',
    marker = dict(  #change the marker style
        size = 12,
        color = 'rgb(51,204,153)',
        symbol = 'pentagon',
        line = dict(
            width = 2,
        )
    )
)]

layout = go.Layout(
    title = 'HDI Vs. Incidence', # Graph title
    xaxis = dict(title = 'HDI'), # x-axis label
    yaxis = dict(title = 'Incidence'), # y-axis label
    hovermode = 'closest' # handles multiple points landing on the same vertical 
)
fig = go.Figure(data=data, layout=layout)
#pyo.plot(fig, filename='scatter.html')
fig.show()

## Creating a Dashboard

In [64]:
# importing libraries
import dash
from dash import dcc
from dash import html
from jupyter_dash import JupyterDash
from dash.dependencies import Input, Output


In [65]:
# we will need to have the latest version of plotly for these code to run properly
# pip install plotly==5.5.0

import plotly
plotly.__version__

'6.5.0'

In [66]:
df_countries3.columns

Index(['Country', 'Incidence', 'Mortality', 'Code2', 'Code3', 'Continent',
       'Human Development Index (HDI)', 'Life Expectancy at Birth',
       'Expected Years of Schooling', 'Mean Years of Schooling',
       'Gross National Income (GNI) per Capita',
       'GNI per capita rank minus HDI rank',
       'National HPV Vaccination Program', 'Sub-Region'],
      dtype='object')

In [67]:
pio.templates.default = "plotly_white"

x_features = ['Human Development Index (HDI)', 'Life Expectancy at Birth',
       'Expected Years of Schooling', 'Mean Years of Schooling',
       'Gross National Income (GNI) per Capita', 'Incidence', 'Mortality']

y_features = ['Incidence', 'Mortality']

markdown_text = '''


*Incidence* = Age-Standardized Incidence Rate per 100,000 in 2020  |  *Mortality* = Age-Standardized Mortality Rate per 100,000 in 2020

*National HPV Vaccination Programs* = Countries which had national HPV vaccination programs as of 17th August 2021. 

Data Sources: Global Cancer Observatory (CC rates),  path.org (list of national HPV vaccine programs) and United Nations Development Programme (HDI and component indicators)



'''

app = JupyterDash(__name__)

# Create server variable with Flask server object for use with gunicorn
server = app.server


########################### Dashboard Layout ################################################################################
app.layout = html.Div([
        html.Div([html.H1('Global Inequalities in Cervical Cancer Rates',style={
                                                             'text-align': 'center',
                                                            'margin-top': '10px', 'margin-bottom': '10px', 'color': '#570A8C'})]),


        html.Div(children=[
            html.Div(children=[
                dcc.RadioItems(id='main_ind', options=[{'label': i.title(), 'value': i} for i in y_features], value='Incidence'),]
                    , style={'width': '45%', 'padding': 10, 'text-align': 'right','color': '#570A8C', 'display': 'inline-block','font-size': '20px'}),                                     
        
            html.Div(children=[
                dcc.RadioItems(id='geo_ind', options=[
                                {'label': 'Continent', 'value': 'Continent'},
                                {'label': 'Sub-Region', 'value': 'Sub-Region'}], 
                                value='Continent'),]
                    , style={'width': '40%', 'padding': 10, 'margin-left': '60px', 'text-align': 'left','color': '#570A8C', 'display': 'inline-block','font-size': '20px'}),                                     
             ]),                                                             
    

        html.Div(children=[
            html.Div(children=[
                    dcc.Graph(id='sunburst-chart',style={'border': '0px solid #cfcfcf'}, config={'displayModeBar': False})
            ],style={'text-align': 'center','display': 'inline-block','width': '40%'}),
                         
                             
         
            html.Div(children=[ 
                    dcc.Graph(id='bar-graph',style={'border': '0px solid #cfcfcf'}),   
            ],style={'text-align': 'center','display': 'inline-block','width': '60%'} ),  
                
                       
        ]),
    

        html.Div(children=[

            html.Div(children=[
               
                dcc.Graph(id='map-chart', style={'border': '0px solid #cfcfcf'}),
                           
                ],style={'display': 'inline-block','width': '48%'}),
                             
            html.Div(children=[

                        dcc.Graph(id='scatter-plot',style={'border': '0px solid #cfcfcf'}),

                        dcc.Dropdown(id='xaxis', options=[{'label': i.title(), 'value': i} for i in x_features], 
                                 value='Human Development Index (HDI)', style={'text-align': 'center','display': 'inline-block','width': '100%'}),
            ], 
                           style={'width': '50%', 'display': 'inline-block'}),
            
            
         ]),    
    
    
    
        html.Div(children=[
            dcc.Markdown(markdown_text)
        ], style={'text-align': 'center'}),   
    
    
             ],style={'border': '1px solid #cfcfcf'})

########################### Callbacks for Interactive Graphs ###############################################################################


# Fig. 1 Sunburst plot

@app.callback(
    Output('sunburst-chart', 'figure'),
    [Input('geo_ind', 'value'),
    Input('main_ind', 'value')])
def update_sun(geo_id, main_ind):
    fig = px.sunburst(df_countries3, 
                  path=[geo_id, 'Country'],
                  color=df_countries3[main_ind], 
                  values=df_countries3[main_ind],
                 color_continuous_scale="bluered",
                 color_continuous_midpoint=np.average(df_countries3[main_ind]),
                   hover_name=main_ind, hover_data={main_ind:':.2f'}
                     )
    fig.update_layout(title_text='Click on {} for {} Rates by Country'.format(geo_id.title(), main_ind.title()), title_x=0.5)
    fig.update_layout(margin={"r":0,"t":80,"l":0,"b":0})
    fig.update_coloraxes(showscale=False) 
    
    return fig
 
# Fig. 2 Bar Graph   
    
@app.callback(
    Output('bar-graph', 'figure'),
    [Input('geo_ind', 'value'),
    Input('main_ind', 'value')])
def update_bar(geo_ind, main_ind):
    fig = px.bar(df_countries3.groupby(geo_ind).mean()[main_ind], 
                color=df_countries3.groupby(geo_ind).mean()[main_ind],
               color_continuous_scale="bluered",
               opacity=0.6,
                text_auto='.1f',
                )
    fig.update_layout(title_text='{} Rates by {}'.format(main_ind.title(), geo_ind.title()), title_x=0.5)
    fig.update_xaxes(categoryorder='total ascending')
    fig.update_xaxes(tickangle=45)
    fig.update_coloraxes(showscale=False)
    fig.update_layout(margin={"r":0,"t":80,"l":0,"b":0})
    fig.update_layout(xaxis=dict(tickfont=dict(size=10), title=''), yaxis=dict(title='Average {} Rate'.format(main_ind.title())))
    
    return fig


# Fig 3. Map

@app.callback(
    Output('map-chart', 'figure'),
    [Input('main_ind', 'value')])
def update_map(main_ind):
    fig = px.scatter_geo(df_countries3,
            locations="Code3",
            color=main_ind,
            size=main_ind,
            color_continuous_scale="bluered",
            hover_name="Country", hover_data={main_ind:True, 'National HPV Vaccination Program':True, 'Sub-Region':True, 'Code3':False}
                )
        
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    return fig  




# Fig. 4 Scatter Plot  
    

@app.callback(
    Output('scatter-plot', 'figure'),
    [Input('xaxis', 'value'),
     Input('main_ind', 'value')])
def update_graph(xaxis_name, main_ind):
        fig = px.scatter(df_countries3,
            x=xaxis_name,
            y=main_ind,
            color=df_countries3['National HPV Vaccination Program'],
            size=main_ind,
            color_discrete_map={"Yes": "MediumPurple", "No": "Red"},
            opacity=0.7,
            trendline='ols',
            hover_name="Country", hover_data={main_ind:True, xaxis_name:True}
        )
        
        fig.update_layout(title_text='{} Vs. {} (click x-axis to change)'.format(main_ind.title(), xaxis_name.title()), title_x=0.5)        
        fig.update_layout(margin={"r":20,"t":100,"l":0,"b":0})
        fig.update_layout(legend=dict(
                title="National HPV Vaccination Program?",
                yanchor="top",
                y=0.99,
                xanchor="right",
                x=0.99))
        fig.update_layout(yaxis=dict(tickfont=dict(size=10)))
        fig.update_layout(xaxis=dict(tickfont=dict(size=10), title=''))
              
        return fig



/workspaces/Global-Air-Quality-Dashboard/.venv/lib/python3.12/site-packages/dash/dash.py:642: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.



In [68]:
import socket
h_name = socket.gethostname()
IP_address = socket.gethostbyname(h_name)

In [69]:
# Run the AR dashboard on port 8052 so it can run alongside the FINAL notebook (port 8051)
app.run(host=IP_address, port=8052, jupyter_mode='external')

Dash app running on http://127.0.0.1:8052/
